In [40]:
import tensorflow as tf
from tensorflow.keras.models import load_model # type: ignore
import pickle
import numpy as np
import pandas as pd

In [41]:
#load the model scaler and encorder

In [42]:
model = load_model('./churn_model.h5')

with open("label_encorder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encorder_geo.pkl", "rb") as file:
    onehot_encorder_geo = pickle.load(file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

In [43]:
#dummy input data

input_data = {
    
    'CreditScore': 6.00,
    'Geography': "France",
    'Gender' : 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 6000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary' : 50000

}

In [44]:
data = pd.DataFrame([input_data])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,6.0,France,Male,40,3,6000,2,1,1,50000


In [45]:
data["Gender"] = label_encoder_gender.transform(data["Gender"])

In [46]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,6.0,France,1,40,3,6000,2,1,1,50000


In [47]:
geo_encorder = onehot_encorder_geo.transform(data[["Geography"]])
geo_encorder

array([[1., 0., 0.]])

In [48]:
geo_encoded_df=pd.DataFrame(geo_encorder, columns=onehot_encorder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [49]:
data = pd.concat([data.drop(["Geography"], axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,6.0,1,40,3,6000,2,1,1,50000,1.0,0.0,0.0


In [50]:
data = scaler.transform(data)
data

array([[-6.7003574 ,  0.91324755,  0.10479359, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [51]:
#make prediction

prediction = model.predict(data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step


array([[0.02756643]], dtype=float32)

In [52]:
prediction[0][0]

0.027566435

In [53]:
if prediction[0][0] > 0.5:
    print("Customer will churn")
else:
    print("Customer will not churn")

Customer will not churn
